In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import sys
import taurex.log
taurex.log.disableLogging()

from taurex.model import TransmissionModel
from taurex.planet import Planet
from taurex.stellar import BlackbodyStar
from taurex.chemistry import TaurexChemistry, ConstantGas
from taurex.temperature import Guillot2010, Isothermal
from taurex.contributions import RayleighContribution
from taurex.model import TransmissionModel
from taurex.contributions import AbsorptionContribution
from taurex.optimizer.nestle import NestleOptimizer
from taurex.cache import OpacityCache,CIACache
from taurex.data.spectrum.observed import ObservedSpectrum



In [3]:
# We set the path with the necessary folders

OpacityCache().clear_cache()
OpacityCache().set_opacity_path("/home/fabrizio/Desktop/test_retrieval/atmosphere-20241205T075608Z-001/atmosphere/xsecs")
CIACache().set_cia_path("/home/fabrizio/Desktop/test_retrieval/atmosphere-20241205T075608Z-001/atmosphere/cia")


In [4]:
# Define the Stellar parameters
star_radius = 0.67    # Solar radii
T_star = 4425.0       # Stellar temperature (K)

# Create a planet instance 
planet = Planet()

# Create a star instance
star = BlackbodyStar(temperature=T_star, radius=star_radius)

# Create temperature profile
guillot = Guillot2010()
isothermal = Isothermal()

In [5]:
# Initializing the chemistry
chemistry = TaurexChemistry()
# Define gasses for H2O, CH4, CO2, and CO
gases = ['H2O', 'CH4', 'CO2', 'CO']

for gas_name in gases:
    chemistry.addGas(ConstantGas(gas_name))

In [6]:
# Initialize the transmission model
tm = TransmissionModel(planet=planet, chemistry=chemistry, star=star, atm_min_pressure=1e-0, atm_max_pressure=1e6, nlayers=30, temperature_profile=isothermal)

In [7]:
# We add the Rayleigh and absorption contributions
tm.add_contribution(RayleighContribution())
tm.add_contribution(AbsorptionContribution())

In [ ]:
# We build and run the model
tm.build()
tm.model()

In [ ]:
# We import the model we want to fit and we plot it to confirm
obs = ObservedSpectrum('/home/fabrizio/Desktop/computational_astrophyisics/comp_astro_24/assignment_3/TaskB/WASP-107-b_spectrum_assignment3_taskB.dat')

plt.figure()
plt.errorbar(obs.wavelengthGrid, obs.spectrum, obs.errorBar, label='Obs')
plt.title('Uploaded spectrum')
#plt.xscale('log')
plt.xlabel('log($\mu$m)')
plt.ylabel('Absorbed relative flux')
plt.grid()
plt.legend()
plt.show()

In [42]:
# Initializing the nestle optimizer using a relatively high tolerance and low number of live points
opt = NestleOptimizer(num_live_points=20, method='multi', tol = 5)
opt.set_model(tm)
opt.set_observed(obs)

In [ ]:
# We plot the possible fitting parameters so we can choose
list(tm.fittingParameters.keys())

In [44]:
# Initializing the parameters we want to 
opt.enable_fit('T')
opt.set_boundary('T',[100, 1500])

opt.enable_fit('planet_radius')
opt.set_boundary('planet_radius',[0.5, 1.5])

opt.enable_fit('H2O')
opt.set_boundary('H2O', [1e-9, 1e-1])

opt.enable_fit('CO2')
opt.set_boundary('CO2', [1e-9, 1e-1])

opt.enable_fit('CH4')
opt.set_boundary('CH4', [1e-9, 1e-1])

opt.enable_fit('CO')
opt.set_boundary('CO', [1e-9, 1e-1])


In [ ]:
# Finding the best fit
solution = opt.fit()
taurex.log.disableLogging()

In [ ]:
# Plotting the resulting best fit with residuals

import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

obin = obs.create_binner()

for solution, optimized_map, optimized_value, values in opt.get_solution():
    # Update the model with the optimized parameters
    opt.update_model(optimized_map)

    # Compute the binned model spectrum
    model_spectrum = obin.bin_model(tm.model(obs.wavenumberGrid))[1]

    # Calculate residuals
    residuals = obs.spectrum - model_spectrum

    # Create a figure with gridspec for subplot size control
    plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])  # Top subplot 3x larger than the bottom

    # Top panel: Observed vs. Retrieved spectrum
    ax1 = plt.subplot(gs[0])
    ax1.errorbar(obs.wavelengthGrid, obs.spectrum, yerr=obs.errorBar, label='Observed Spectrum', capsize=2, elinewidth=0.5)
    ax1.plot(obs.wavelengthGrid, model_spectrum, label='Retrieved Model', color='r')
    ax1.set_xscale('log')
    plt.ylabel('Absorbed relative flux')
    ax1.set_title('Observed spectrum and Retrieved Model')
    #ax1.set_xlim([2.2, 5.8])
    ax1.set_ylim([0.0191, 0.0217])
    ax1.legend()
    ax1.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Bottom panel: Residuals
    ax2 = plt.subplot(gs[1])
    ax2.errorbar(obs.wavelengthGrid, residuals, yerr=obs.errorBar, label='Residuals', capsize=2, elinewidth=0.5)
    ax2.set_xscale('log')
    ax2.set_ylabel('Residuals')
    ax2.set_ylim([-0.001, 0.001])
    #ax2.set_xlim([2.2, 5.8])
    ax2.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Adjust layout and show the plots
    plt.tight_layout()
    plt.xlabel('log($\mu$m)')
    
    plt.show()


In [68]:
# Saving the values 
optimized_map_model= optimized_map
values = opt.get_samples(solution)
np.savetxt('retrieval_values.txt', values)
print(optimized_map)

In [ ]:
import corner
import numpy as np
import matplotlib.lines as mlines

# Load data
values = (np.genfromtxt('values_wasp107b.txt', dtype='float', usecols=(0, 1, 2, 3, 4, 5), unpack=True)).T

# Create corner plot
figure = corner.corner(values, labels=["Radius", "T", "H$_2$O", r"CH$_4$", r"CO$_2$", "CO"], show_titles=False, smooth=1)

# Overlay lines only on the 1D histograms 
true_values = [0.98, 1315, -3.95, -5.78, -2.57, -4.81]
map_estimate = [0.93, 2401, -4.64, -5.85, -3.14, -7.49]  

# Calculate percentiles for errors
percentiles = np.percentile(values, [16, 50, 84], axis=0)
errors_lower = map_estimate - percentiles[0]  # 16th percentile as lower error
errors_upper = percentiles[2] - map_estimate # 84th percentile as upper error

# Iterate over diagonal axes
axes = figure.axes
ndim = len(true_values)
for i in range(ndim):
    ax = axes[i * (ndim + 1)]  # Diagonal subplot index
    
    # Add vertical line for true values
    ax.axvline(true_values[i], color="C2", linestyle='--', label="True Value" if i == 0 else "")
    
    # Add vertical line for MAP estimates
    ax.axvline(map_estimate[i], color="C3", linestyle='--', label="MAP Estimate" if i == 0 else "")

    # Add text annotation for MAP estimate with 84th percentile
    text = (f"{['Radius', 'T', 'H$_2$O', 'CH$_4$', 'CO$_2$', 'CO'][i]} = " f"${map_estimate[i]:.2f}^{{+{errors_upper[i]:.2f}}}_{{-{errors_lower[i]:.2f}}}$")

    ax.text(0.5, 1.05, text, transform=ax.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='center', color="black")

# Create a custom legend
true_line = mlines.Line2D([], [], color="C2", linestyle="--", label="True Values")
map_line = mlines.Line2D([], [], color="C3", linestyle="--", label="MAP Estimate")
figure.legend(handles=[true_line, map_line], loc="upper right", fontsize=12)

corner.overplot_points(figure, [true_values], marker="*", markersize=12, color="C2", label="True Values")
corner.overplot_points(figure, [map_estimate], marker="*", markersize=12, color="C3", label="MAP Estimate")

# Add title
figure.suptitle("Posteriors", fontsize=16)

# Add a custom legend
import matplotlib.lines as mlines
true_point = mlines.Line2D([], [], color="C2", marker="*", linestyle="None", label="True Values")
map_point = mlines.Line2D([], [], color="C3", marker="*", linestyle="None", label="MAP Estimate")
figure.legend(handles=[true_point, map_point], loc="upper right", fontsize=12)


In [ ]:
# The code we used to compare the retrieval with the two models before using the corner package

# Create a 3x2 grid of subplots
fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 3 rows, 2 columns
axes = axes.flatten()  # Flatten the 2D array of axes into a 1D array for easy indexing

# Loop over each pair of datasets and corresponding subplot
ax = axes[i]

# Plot the histograms with 'step' style
axes[0].hist(values_mcmc[:,0], bins=30, histtype='step', color='red', label='classic')
axes[0].hist(values_multi[:,0], bins=30, histtype='step', color='blue', label='multi')
axes[0].axvline(1.27, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[0].set_title(f'Posterior of planet radius')
axes[0].set_xlabel('Value [jupiter radii]')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, linestyle='--', linewidth=0.5)

axes[1].hist(values_mcmc[:,1], bins=30, histtype='step', color='red', label='classic')
axes[1].hist(values_multi[:,1], bins=30, histtype='step', color='blue', label='multi')
axes[1].axvline(1200, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[1].set_title(f'Posterior of irradiated temperature')
axes[1].set_xlabel('Value [K]')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, linestyle='--', linewidth=0.5)

axes[2].hist(values_mcmc[:,2], bins=30, histtype='step', color='red', label='classic')
axes[2].hist(values_multi[:,2], bins=30, histtype='step', color='blue', label='multi')
axes[2].axvline(-3.95, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[2].set_title(f'Posterior of H$_2$O abundance')
axes[2].set_xlabel('Value [$\log_{10}$(abundance)]')
axes[2].set_ylabel('Frequency')
axes[2].legend()
axes[2].grid(True, linestyle='--', linewidth=0.5)

axes[3].hist(values_mcmc[:,3], bins=30, histtype='step', color='red', label='classic')
axes[3].hist(values_multi[:,3], bins=30, histtype='step', color='blue', label='multi')
axes[3].axvline(-5.78, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[3].set_title(f'Posterior of CH$_4$ abundance')
axes[3].set_xlabel('Value [$\log_{10}$(abundance)]')
axes[3].set_ylabel('Frequency')
axes[3].legend()
axes[3].grid(True, linestyle='--', linewidth=0.5)

axes[4].hist(values_mcmc[:,4], bins=30, histtype='step', color='red', label='classic')
axes[4].hist(values_multi[:,4], bins=30, histtype='step', color='blue', label='multi')
axes[4].axvline(-2.57, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[4].set_title(f'Posterior of CO$_2$ abundance')
axes[4].set_xlabel('Value [$\log_{10}$(abundance)]')
axes[4].set_ylabel('Frequency')
axes[4].legend()
axes[4].grid(True, linestyle='--', linewidth=0.5)

axes[5].hist(values_mcmc[:,5], bins=30, histtype='step', color='red', label='classic')
axes[5].hist(values_multi[:,5], bins=30, histtype='step', color='blue', label='multi')
axes[5].axvline(-4.81, label = 'true value', color = 'g', linewidth = 3, linestyle = '--')
axes[5].set_title(f'Posterior of CO abundance')
axes[5].set_xlabel('Value [$\log_{10}$(abundance)]')
axes[5].set_ylabel('Frequency')
axes[5].legend()
axes[5].grid(True, linestyle='--', linewidth=0.5)

# Adjust layout for better spacing
plt.tight_layout()

# Show the final plot
plt.show()